# Reliability calculations with arrays as arguments in the model

This example demonstrates how to perform reliability calculations with a limit state function that has multiple arguments, all following the same distribution function.

### Define model

First, let's import the necessary package:

In [1]:
from probabilistic_library import ReliabilityProject, DistributionType, ReliabilityMethod, StandardNormal

We consider the following limit state function:

$Z = 1.9 - [\sum_{i=1}^{n}(a_i)+\sum_{j=1}^{m}(b_j)]$

This is a linear model involving two arrays, $a$ and $b$.

In [2]:
def linear(const_val: float, a: list[float], b:list[float]):
    return const_val + sum(a) + sum(b)

To perform a reliability analysis, we create a reliability project and specify the limit state function (model):

In [3]:
project = ReliabilityProject()
project.model = linear

We assume that array $a$ consists of $25$ variables following a uniform distribution over the interval $[-1, 1]$. Array $b$ consists of $30$ variables, each normally distributed with a mean of $0$ and a standard deviation of $1$. This can be defined as follows:

In [4]:
project.variables["const_val"].distribution = DistributionType.deterministic
project.variables["const_val"].is_array = False
project.variables["const_val"].mean = 1.9

project.variables["a"].distribution = DistributionType.uniform
project.variables["a"].is_array = True # states that a is an array
project.variables["a"].array_size = 25
project.variables["a"].minimum = -1
project.variables["a"].maximum = 1

project.variables["b"].distribution = DistributionType.normal
project.variables["b"].is_array = True # states that b is an array
project.variables["b"].array_size = 30
project.variables["b"].mean = 0
project.variables["b"].deviation = 1

### Perform reliability calculations

We perform the reliability calculations using the form method.

In [5]:
project.settings.reliability_method = ReliabilityMethod.form
project.settings.relaxation_factor = 0.15
project.settings.maximum_iterations = 50
project.settings.variation_coefficient = 0.02
project.run()

dp = project.design_point

def read_results(dp):

    beta = dp.reliability_index

    print(f"Beta = {beta}")

    pf = StandardNormal.get_q_from_u(beta)
    print(f"Probability of failure = {pf}")

    for alpha in dp.alphas:
        print(f"{alpha.variable.name}: alpha = {alpha.alpha}, x = {alpha.x}")

    if dp.is_converged:
        print(f"Converged (convergence = {dp.convergence} < {project.settings.variation_coefficient})")
    else:
        print(f"Not converged (convergence = {dp.convergence} > {project.settings.variation_coefficient})")
        
    print(f"Model runs = {dp.total_model_runs}")

read_results(project.design_point)

Beta = 0.2804156772851438
Probability of failure = 0.3895793053058858
const_val: alpha = 0.0, x = 1.9
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.02630909543259996
a: alpha = 0.11760946804193541, x = -0.0263